In [1]:
%matplotlib inline


# Double FHE with BFV scheme

Integer FHE Demo for Pyfhel, covering all the posibilities offered by Pyfhel
regarding the BFV scheme (see https://eprint.iacr.org/2012/144.pdf).


In [2]:
import numpy as np
from Pyfhel import Pyfhel

## 1. BFV context and key setup
We take a look at the different parameters that can be set for the BFV scheme.



In [3]:
HE = Pyfhel()           # Creating empty Pyfhel object
bfv_params = {
    'scheme': 'BFV',    # can also be 'bfv'
    'n': 2**13,         # Polynomial modulus degree, the num. of slots per plaintext,
                        #  of elements to be encoded in a single ciphertext in a
                        #  2 by n/2 rectangular matrix (mind this shape for rotations!)
                        #  Typ. 2^D for D in [10, 16]
    't': 65537,         # Plaintext modulus. Encrypted operations happen modulo t
                        #  Must be prime such that t-1 be divisible by 2^N.
    't_bits': 20,       # Number of bits in t. Used to generate a suitable value 
                        #  for t. Overrides t if specified.
    'sec': 128,         # Security parameter. The equivalent length of AES key in bits.
                        #  Sets the ciphertext modulus q, can be one of {128, 192, 256}
                        #  More means more security but also slower computation.
}
HE.contextGen(**bfv_params)  # Generate context for bfv scheme
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()       # Rotate key generation --> Allows rotation/shifting
HE.relinKeyGen()        # Relinearization key generation

print("\n1. Pyfhel FHE context generation")
print(f"\t{HE}")


1. Pyfhel FHE context generation
	<bfv Pyfhel obj at 0x7f53ec3392b0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=8192, t=1032193, sec=128, qi=[], scale=1.0, )]>


## 2. Double Array Encoding & Encryption
we will define two 1D double arrays, encode and encrypt them:
arr1 = [0, 1, ... n-1] (length n)
arr2 = [-t//2, -1, 1]  (length 3) --> Encoding fills the rest of the array with zeros



In [4]:
arr1 = np.array([1.0, 2.0, 3.0], dtype=np.double)    # Max possible value is t/2-1. Always use type int64!
arr2 = np.array([2.0, 3.0, 4.0], dtype=np.double)    # Min possible value is -t/2. 

ptxt1 = HE.encode(arr1)   # Creates a PyPtxt plaintext with the encoded arr1
ptxt2 = HE.encode(arr2)   # plaintexts created from arrays shorter than 'n' are filled with zeros.

ctxt1 = HE.encryptPtxt(ptxt1) # Encrypts the plaintext ptxt1 and returns a PyCtxt
ctxt2 = HE.encryptPtxt(ptxt2) #  Alternatively you can use HE.encryptInt(arr2)

# Otherwise, a single call to `HE.encrypt` would detect the data type,
#  encode it and encrypt it
#> ctxt1 = HE.encrypt(arr1)

print("\n2. Double Encoding & Encryption, ")
print("->\tarr1 ", arr1,'\n\t==> ptxt1 ', ptxt1,'\n\t==> ctxt1 ', ctxt1)
print("->\tarr2 ", arr2,'\n\t==> ptxt2 ', ptxt2,'\n\t==> ctxt2 ', ctxt2)


2. Double Encoding & Encryption, 
->	arr1  [1. 2. 3.] 
	==> ptxt1  <Pyfhel Plaintext at 0x7f53ec267a80, scheme=bfv, poly=3C6DDx^8191 + C0707x^8190..., is_ntt=-> 
	==> ctxt1  <Pyfhel Ciphertext at 0x7f53ec1f0f90, scheme=bfv, size=2/2, noiseBudget=146>
->	arr2  [2. 3. 4.] 
	==> ptxt2  <Pyfhel Plaintext at 0x7f53ebd48900, scheme=bfv, poly=7338Dx^8191 + 2036Fx^8190..., is_ntt=-> 
	==> ctxt2  <Pyfhel Ciphertext at 0x7f53ebd4b860, scheme=bfv, size=2/2, noiseBudget=146>


## 3. Securely operating on encrypted double arrays
We try all the operations supported by Pyfhel.
 Note that, to operate, the ciphertexts/plaintexts must be built with the same
 context. Internal checks prevent ops between ciphertexts of different contexts.



In [5]:
# Ciphertext-ciphertext ops:
ccSum = ctxt1 + ctxt2       # Calls HE.add(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 += ctxt2` for inplace operation
ccSub = ctxt1 - ctxt2       # Calls HE.sub(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 -= ctxt2` for inplace operation
ccMul = ctxt1 * ctxt2       # Calls HE.multiply(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 *= ctxt2` for inplace operation
cSq   = ctxt1**2            # Calls HE.square(ctxt1, in_new_ctxt=True)
                            #  `ctxt1 **= 2` for inplace operation
cNeg  = -ctxt1              # Calls HE.negate(ctxt1, in_new_ctxt=True)
                            # 
cPow  = ctxt1**3            # Calls HE.power(ctxt1, 3, in_new_ctxt=True)
                            #  `ctxt1 **= 3` for inplace operation
cRotR = ctxt1 >> 2          # Calls HE.rotate(ctxt1, k=2, in_new_ctxt=True)
                            #  `ctxt1 >>= 2` for inplace operation
                            # WARNING! the encoded data is placed in a n//2 by 2
                            #  matrix. Hence, these rotations apply independently
                            #  to each of the rows!
cRotL = ctxt1 << 2          # Calls HE.rotate(ctxt1, k=-2, in_new_ctxt=True)
                            #  `ctxt1 <<= 2` for inplace operation

# Ciphetext-plaintext ops
cpSum = ctxt1 + ptxt2       # Calls HE.add_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 += ctxt2` for inplace operation
cpSub = ctxt1 - ptxt2       # Calls HE.sub_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 -= ctxt2` for inplace operation
cpMul = ctxt1 * ptxt2       # Calls HE.multiply_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 *= ctxt2` for inplace operation


print("3. Secure operations")
print(" Ciphertext-ciphertext: ")
print("->\tctxt1 + ctxt2 = ccSum: ", ccSum)
print("->\tctxt1 - ctxt2 = ccSub: ", ccSub)
print("->\tctxt1 * ctxt2 = ccMul: ", ccMul)
print(" Single ciphertext: ")
print("->\tctxt1**2      = cSq  : ", cSq  )
print("->\t- ctxt1       = cNeg : ", cNeg )
print("->\tctxt1**3      = cPow : ", cPow )
print("->\tctxt1 >> 2    = cRotR: ", cRotR)
print("->\tctxt1 << 2    = cRotL: ", cRotL)
print(" Ciphertext-plaintext: ")
print("->\tctxt1 + ptxt2 = cpSum: ", cpSum)
print("->\tctxt1 - ptxt2 = cpSub: ", cpSub)
print("->\tctxt1 * ptxt2 = cpMul: ", cpMul)

3. Secure operations
 Ciphertext-ciphertext: 
->	ctxt1 + ctxt2 = ccSum:  <Pyfhel Ciphertext at 0x7f53ec1646d0, scheme=bfv, size=2/2, noiseBudget=145>
->	ctxt1 - ctxt2 = ccSub:  <Pyfhel Ciphertext at 0x7f53ebd590e0, scheme=bfv, size=2/2, noiseBudget=146>
->	ctxt1 * ctxt2 = ccMul:  <Pyfhel Ciphertext at 0x7f53ebd59130, scheme=bfv, size=3/3, noiseBudget=114>
 Single ciphertext: 
->	ctxt1**2      = cSq  :  <Pyfhel Ciphertext at 0x7f53ebd59180, scheme=bfv, size=3/3, noiseBudget=114>
->	- ctxt1       = cNeg :  <Pyfhel Ciphertext at 0x7f53ebd59220, scheme=bfv, size=2/2, noiseBudget=146>
->	ctxt1**3      = cPow :  <Pyfhel Ciphertext at 0x7f53ebd59270, scheme=bfv, size=2/2, noiseBudget=82>
->	ctxt1 >> 2    = cRotR:  <Pyfhel Ciphertext at 0x7f53ebd592c0, scheme=bfv, size=2/2, noiseBudget=142>
->	ctxt1 << 2    = cRotL:  <Pyfhel Ciphertext at 0x7f53ebd59310, scheme=bfv, size=2/2, noiseBudget=143>
 Ciphertext-plaintext: 
->	ctxt1 + ptxt2 = cpSum:  <Pyfhel Ciphertext at 0x7f53ec146540, scheme=bfv, s

## 4. BFV Relinearization: What, why, when
Ciphertext-ciphertext multiplications increase the size of the polynoms 
 representing the resulting ciphertext. To prevent this growth, the 
 relinearization technique is used (typically right after each c-c mult) to 
 reduce the size of a ciphertext back to the minimal size (two polynoms c0 & c1).
 For this, a special type of public key called Relinearization Key is used.

In Pyfhel, you can either generate a relin key with HE.RelinKeyGen() or skip it
 and call HE.relinearize() directly, in which case a warning is issued.

Note that HE.power performs relinearization after every multiplication.



In [6]:
print("\n4. Relinearization-> Right after each multiplication.")
print(f"ccMul before relinearization (size {ccMul.size()}): {ccMul}")
~ccMul    # Equivalent to HE.relinearize(ccMul). Relin always happens in-place.
print(f"ccMul after relinearization (size {ccMul.size()}): {ccMul}")
print(f"cPow after 2 mult&relin rounds:  (size {cPow.size()}): {cPow}")


4. Relinearization-> Right after each multiplication.
ccMul before relinearization (size 3): <Pyfhel Ciphertext at 0x7f53ebd59130, scheme=bfv, size=3/3, noiseBudget=114>
ccMul after relinearization (size 2): <Pyfhel Ciphertext at 0x7f53ebd59130, scheme=bfv, size=2/3, noiseBudget=114>
cPow after 2 mult&relin rounds:  (size 2): <Pyfhel Ciphertext at 0x7f53ebd59270, scheme=bfv, size=2/2, noiseBudget=82>


## 5. Decrypt & Decode results
Time to decrypt results! We use HE.decryptInt for this. 
 HE.decrypt() could also be used, in which case the decryption type would be
 inferred from the ciphertext metadata. 



In [7]:
r1     = HE.decrypt(ctxt1)
r2     = HE.decrypt(ctxt2)
rccSum = HE.decrypt(ccSum)
rccSub = HE.decrypt(ccSub)
rccMul = HE.decrypt(ccMul)
rcSq   = HE.decrypt(cSq  )
rcNeg  = HE.decrypt(cNeg )
rcPow  = HE.decrypt(cPow )
rcRotR = HE.decrypt(cRotR)
rcRotL = HE.decrypt(cRotL)
rcpSum = HE.decrypt(cpSum)
rcpSub = HE.decrypt(cpSub)
rcpMul = HE.decrypt(cpMul)

print("5. Decrypting results")
print(" Original ciphertexts: ")
print("   ->\tctxt1 --(decr)--> ", r1)
print("   ->\tctxt2 --(decr)--> ", r2)
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt1 + ctxt2 = ccSum --(decr)--> ", rccSum)
print("   ->\tctxt1 - ctxt2 = ccSub --(decr)--> ", rccSub)
print("   ->\tctxt1 * ctxt2 = ccMul --(decr)--> ", rccMul)
print(" Single ciphertext: ")
print("   ->\tctxt1**2      = cSq   --(decr)--> ", rcSq  )
print("   ->\t- ctxt1       = cNeg  --(decr)--> ", rcNeg )
print("   ->\tctxt1**3      = cPow  --(decr)--> ", rcPow )
print("   ->\tctxt1 >> 2    = cRotR --(decr)--> ", rcRotR)
print("   ->\tctxt1 << 2    = cRotL --(decr)--> ", rcRotL)
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt1 + ptxt2 = cpSum --(decr)--> ", rcpSum)
print("   ->\tctxt1 - ptxt2 = cpSub --(decr)--> ", rcpSub)
print("   ->\tctxt1 * ptxt2 = cpMul --(decr)--> ", rcpMul)

5. Decrypting results
 Original ciphertexts: 
   ->	ctxt1 --(decr)-->  [1 2 3 ... 0 0 0]
   ->	ctxt2 --(decr)-->  [2 3 4 ... 0 0 0]
 Ciphertext-ciphertext Ops: 
   ->	ctxt1 + ctxt2 = ccSum --(decr)-->  [3 5 7 ... 0 0 0]
   ->	ctxt1 - ctxt2 = ccSub --(decr)-->  [-1 -1 -1 ...  0  0  0]
   ->	ctxt1 * ctxt2 = ccMul --(decr)-->  [ 2  6 12 ...  0  0  0]
 Single ciphertext: 
   ->	ctxt1**2      = cSq   --(decr)-->  [1 4 9 ... 0 0 0]
   ->	- ctxt1       = cNeg  --(decr)-->  [-1 -2 -3 ...  0  0  0]
   ->	ctxt1**3      = cPow  --(decr)-->  [ 1  8 27 ...  0  0  0]
   ->	ctxt1 >> 2    = cRotR --(decr)-->  [0 0 1 ... 0 0 0]
   ->	ctxt1 << 2    = cRotL --(decr)-->  [3 0 0 ... 0 0 0]
 Ciphertext-plaintext ops: 
   ->	ctxt1 + ptxt2 = cpSum --(decr)-->  [3 5 7 ... 0 0 0]
   ->	ctxt1 - ptxt2 = cpSub --(decr)-->  [-1 -1 -1 ...  0  0  0]
   ->	ctxt1 * ptxt2 = cpMul --(decr)-->  [ 2  6 12 ...  0  0  0]
